In [ ]:
#!pip install pyhwp

In [1]:
import os
from glob import glob
import pandas as pd
#import re

In [2]:
DATA_PATH = os.getcwd()
files = glob(os.path.join(DATA_PATH, '*.hwp'))

In [3]:
files

['C:\\Users\\Kislee\\Workspace\\2023\\고전시가_새\\한국시조대사전.hwp']

In [4]:
f_name = files[0]
!hwp5txt --output hwp.txt "$f_name"

In [5]:
df = pd.read_csv('hwp.txt', delimiter='\n', header=None)

In [6]:
df.head()

,0
0,1
1,가나이다 가이다 小臣 도라 가이다
2,忠臣도 려니와 養親인 마오릿가
3,구틔여 오라 시면 다시 도라 오오리다
4,2


In [7]:
poem_df = pd.DataFrame()

In [8]:
번호 = df[0::4]
초장 = df[1::4]
중장 = df[2::4]
종장 = df[3::4]

In [9]:
번호 = 번호.reset_index(drop=True)
초장 = 초장.reset_index(drop=True)
중장 = 중장.reset_index(drop=True)
종장 = 종장.reset_index(drop=True)

In [10]:
poem_df = pd.concat([번호, 초장, 중장, 종장], axis=1, ignore_index=True)

In [11]:
poem_df.columns=['번호','초장','중장','종장']

In [12]:
poem_df['번호'] = poem_df['번호'].str.replace('.','')
poem_df['번호'] = poem_df['번호'].str.replace(',','')
poem_df['번호'] = poem_df['번호'].str.replace(' ','')

<ipython-input-12-842813b9071e>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  poem_df['번호'] = poem_df['번호'].str.replace('.','')


In [13]:
poem_df['번호'] = poem_df['번호'].astype(int)

In [14]:
poem_df.head()

,번호,초장,중장,종장
0,1,가나이다 가이다 小臣 도라 가이다,忠臣도 려니와 養親인 마오릿가,구틔여 오라 시면 다시 도라 오오리다
1,2, 비 빗긴 람 낫대 멘 뎌 하나바,네 生涯 언마치라 슈고롬도 슈롤샤,生涯를 爲호미 아니라 漁興 계워 하노라
2,3,가노라 가노라 님아 언양 단쳔의 風月江山으로 가노라 님아,가다가 潯陽江의 琵琶聲을 어이 하리,밤즁만 지국총 닷 감 쇼에 잠 못 일워
3,4,가노라 다시 보자 그립거든 어이 살고,비록 千里라타 의야 아니 보랴, 야 겻희 업면 그를 어이 리오
4,5,가노라 白石亭아 보시 보쟈 白鷗들아,너릐 다시 보니 나고쟈 랴만,젹의도 臨鏡亭 이시니 기일가 로라


In [15]:
len(poem_df)

4728

In [16]:
def find_skipped_numbers(column):
    differences = column.diff()
    return differences[differences > 1]

In [17]:
missing_values_A = find_skipped_numbers(poem_df['번호'])
print(missing_values_A)

1391    2.0
1581    2.0
1758    2.0
2641    2.0
3085    2.0
3377    2.0
4094    2.0
4267    2.0
Name: 번호, dtype: float64


In [20]:
#새 사전 가져오기
bird_files = os.path.join(os.getcwd(), '시조 새 사전.xlsx')
print(bird_files)

C:\Users\Kislee\Workspace\2023\고전시가_새\시조 새 사전.xlsx


In [21]:
b_df = pd.read_excel(bird_files)

In [22]:
b_df.head()

,이름,변형
0,가마우지,"가마우지,가마오지"
1,갈매기,"갈매기,백구,갈며기,갈멱이,白鷗,갈메기,白구"
2,감장새,"감장새,감쟝새"
3,거위,鵝
4,공작,"孔雀,공작"


In [23]:
bird_list = []
for birds in b_df['변형']:
    names = birds.split(',')
    for name in names:
        bird_list.append(name)

In [25]:
len(bird_list)

192

In [26]:
def remove_elements_contained_in_others(input_list):
    result = []
    for element in input_list:
        if not any(element in other for other in input_list if element != other):
            result.append(element)
    return result    

In [27]:
def find_matching_value(text):
    founds = []
    for value in bird_list:
        if value in text:
            founds.append(value)
    if len(founds)>0:
        return ','.join(remove_elements_contained_in_others(founds))
    else:
        return None

In [28]:
new_df = pd.DataFrame()

In [29]:
new_df['번호'] = poem_df['번호']
new_df['초장'] = poem_df['초장'].apply(lambda x: find_matching_value(x))
new_df['중장'] = poem_df['중장'].apply(lambda x: find_matching_value(x))
new_df['종장'] = poem_df['종장'].apply(lambda x: find_matching_value(x))

In [30]:
new_df

,번호,초장,중장,종장
0,1,None,None,오리
1,2,None,None,계
2,3,None,None,None
3,4,None,None,None
4,5,白鷗,None,None
...,...,...,...,...
4723,4732,None,None,계
4724,4733,None,鳳凰,None
4725,4734,None,None,None
4726,4735,None,None,None


In [31]:
filtered_df = new_df[new_df['초장'].notnull() | new_df['중장'].notnull() | new_df['종장'].notnull()]

In [34]:
filtered_df

,번호,초장,중장,종장
0,1,None,None,오리
1,2,None,None,계
4,5,白鷗,None,None
8,9,매,None,None
17,18,None,None,오리
...,...,...,...,...
4699,4708,燕,None,None
4701,4710,꾀꼬리,None,None
4714,4723,None,None,계
4723,4732,None,None,계


In [35]:
filtered_df.to_excel('시조별 등장 새.xlsx')